# Tipos de asas e sua geometria

Abaixo estão os três principais tipos de asas definidas em termos de funções, explore-as para compreender como foram traçadas!

Baseando-se nas varíaveis que definem cada tipo de asa, calcula-se algumas grandezas muito importantes no projeto, que são:

* $S_{ref}$ - Área de asa
* $AR$ - Alongamento (Aspect ratio)
* $CMA$ - Corda Média Aerodinâmica (Mean Aerodynamic Chord)

In [2]:
import math
import numpy as np
import bqplot as bq
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from scipy.integrate import quad, trapz
%matplotlib inline

## Asa trapezoidal

A asa trapezoidal é definida por:

* $B$ - Envergadura
* $Cr$ - Corda da raíz (Root chord)
* $\lambda$ - Afilamento (Taper ratio)
* $\Lambda$ - Enflechamento (Sweept angle)
* $cl$ - Linha de centro

A linha de centro não é muito usual, mas adicionamos aqui para demonstrar como a posição de referência do ângulo de enflechamento é importante!

### Exemplo:

![T-27 Tucano](https://www.bravobravoaviation.com/1870-large_default/t-27-tucano-esquadrilha-da-fumaca-demonstration-team-forca-aerea-brasileira.jpg)


In [7]:
Sreftaper= widgets.BoundedFloatText(description='Área de asa [m²]:',format='0.1f',disabled=True)
ARtaper  = widgets.BoundedFloatText(description='Alongamento:',disabled=True)
CMAtaper = widgets.BoundedFloatText(description='CMA:',disabled=True)

def AsaTrapezoidal(B,Croot,taper,sweept,cl):
    
    def Cb(x,b,c):
        return math.pow(np.interp(x,b,c),2)
    
    Nchords = 10
    b = np.linspace(0,B/2,Nchords)
    ctaper = np.linspace(Croot,Croot*taper,Nchords)
    
    I = quad(Cb,0,B/2,args=(b,ctaper))
     
    cltaper=[]
    LEtaper=[]
    TEtaper=[]
    LEtaper.append(0)
    TEtaper.append(Croot)
    cltaper.append(Croot*cl)
    
    for chord,span in zip(ctaper[1:],b[1:]):
        cltaper.append(Croot*cl+span*math.tan(sweept*np.pi/180))
        LEtaper.append(Croot*cl+span*math.tan(sweept*np.pi/180)-chord*cl)
        TEtaper.append(Croot*(cl)+span*math.tan(sweept*np.pi/180)+chord*(1-cl)) 
    
    Sreftaper.value = np.mean(ctaper)*B/2
    ARtaper.value   = (B*B)/Sreftaper.value
    CMAtaper.value  = max(I)/Sreftaper.value
    
    if taper <= 1:
        bcma = np.interp(CMAtaper.value,np.flip(ctaper),np.flip(b))
    else:
        bcma = np.interp(CMAtaper.value,ctaper,b)
    LEcma= np.interp(bcma,b,LEtaper)
    TEcma= np.interp(bcma,b,TEtaper)
    
    fig = plt.figure(figsize=(15, 15), dpi=72, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(2, 1, 1)
    ax.plot(b, LEtaper, color="black")
    ax.plot(b, TEtaper, color="black")
    ax.plot(-b,LEtaper, color="black")
    ax.plot(-b,TEtaper, color="black")  
    ax.plot([B/2,B/2],[LEtaper[-1],TEtaper[-1]], color="black")
    ax.plot([-B/2,-B/2],[LEtaper[-1],TEtaper[-1]], color="black")
    
    ax.plot([bcma, bcma],[LEcma, TEcma], color="red")
    ax.plot(-b,cltaper,color="gray")
    ax.plot(b, cltaper,color="gray")
    ax.grid(True)
    ax.title.set_text('Asa Trapezoidal')
    ax.set_xlabel('Envergadura [m]')
    ax.set_ylabel('Corda [m]')
    ax.set_aspect('equal', 'datalim')
    fig.canvas.draw()
    
interact(AsaTrapezoidal,
        B      = widgets.FloatSlider(min=0.5,max=35.0,step=0.01,value=11.14, description='Envergadura [m]'),
        Croot  = widgets.FloatSlider(min=0.22,max=10.0,step=0.01, value=4.15, description='Corda na raíz[m]'),
        taper  = widgets.FloatSlider(min=0.1,max=2.0, step=0.01,value=0.66, description='Afilamento'),
        sweept = widgets.FloatSlider(min=-15,max=50,step=0.01,value=0,description='Enflechamento [º]'),
        cl     = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.25,description='Linha de referência'),
        );
widgets.HBox([Sreftaper,ARtaper,CMAtaper])

interactive(children=(FloatSlider(value=11.14, description='Envergadura [m]', max=35.0, min=0.5, step=0.01), F…

## Asa Mista

A asa mista é definida por:

* $B$ - Envergadura
* $Cr$ - Corda da raíz
* $\lambda$ - Afilamento
* $\Lambda$ - Enflechamento
* $cl$ - Linha de centro
* $B_{reta}$ - Proporção de asa reta

### Exemplo:

![Cessna 182 Skylane](https://www.top-flite.com/airplanes/topa0906-4view-lg.jpg)


In [8]:
Srefmista= widgets.BoundedFloatText(description='Área de asa [m²]:',format='0.1f',disabled=True)
ARmista  = widgets.BoundedFloatText(description='Alongamento:',disabled=True)
CMAmista = widgets.BoundedFloatText(description='CMA:',disabled=True)

def AsaMista(B,Croot,taper,sweept,cl,cmista):
    
    def Cb(x,b,c):
        return math.pow(np.interp(x,b,c),2)
    
    Nchords = 10
    b = np.linspace(0,B/2,Nchords-1)
    b = np.append(b,cmista*B/2)
    b = np.sort(b)
    
    c = []    
    
    for bi in b:
        if bi <= cmista*B/2:
            c.append(Croot)
        else:
            if taper <= 1:
                c.append(np.interp(bi,[cmista*B/2, B/2],[Croot, Croot*taper]))
            else:
                c.append(np.interp(bi,np.flip([cmista*B/2, B/2]),np.flip([Croot, Croot*taper])))
    
    I = quad(Cb,0,B/2,args=(b,c))
     
    clmista=[]
    LEmista=[]
    TEmista=[]
    LEmista.append(0)
    TEmista.append(Croot)
    clmista.append(Croot*cl)

    
    for chord,span in zip(c[1:],b[1:]):
        clmista.append(Croot*cl+span*math.tan(sweept*np.pi/180))
        LEmista.append(Croot*cl+span*math.tan(sweept*np.pi/180)-chord*cl)
        TEmista.append(Croot*(cl)+span*math.tan(sweept*np.pi/180)+chord*(1-cl)) 
    
    Srefmista.value = Croot*cmista*B/2 + (Croot+Croot*taper)/2*(B/2*(1-cmista))
    ARmista.value   = (B*B)/Srefmista.value
    CMAmista.value  = max(I)/Srefmista.value
    
    if taper <= 1:
        bcma = np.interp(CMAmista.value,np.flip(c),np.flip(b))
    else:
        bcma = np.interp(CMAmista.value,c,b)
    LEcma= np.interp(bcma,b,LEmista)
    TEcma= np.interp(bcma,b,TEmista)
    
    fig = plt.figure(figsize=(15, 15), dpi=72, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(2, 1, 1)
    ax.plot(b, LEmista, color="black")
    ax.plot(b, TEmista, color="black")
    ax.plot(-b,LEmista, color="black")
    ax.plot(-b,TEmista, color="black")  
    ax.plot([B/2,B/2],[LEmista[-1],TEmista[-1]], color="black")
    ax.plot([-B/2,-B/2],[LEmista[-1],TEmista[-1]], color="black")
    
    ax.plot([bcma, bcma],[LEcma, TEcma], color="red")
    ax.plot(-b,clmista,color="gray")
    ax.plot(b, clmista,color="gray")
    ax.grid(True)
    ax.title.set_text('Asa mista')
    ax.set_xlabel('Envergadura [m]')
    ax.set_ylabel('Corda [m]')
    ax.set_aspect('equal', 'datalim')
    fig.canvas.draw()
    
interact(AsaMista,
        B      = widgets.FloatSlider(min=0.5,max=35.0,step=0.01,value=11, description='Envergadura [m]'),
        Croot  = widgets.FloatSlider(min=0.22,max=10.0,step=0.01, value=3.2, description='Corda na raíz[m]'),
        taper  = widgets.FloatSlider(min=0.1,max=2.0, step=0.01,value=0.66, description='Afilamento'),
        sweept = widgets.FloatSlider(min=-15,max=50,step=0.01,value=0,description='Enflechamento [º]'),
        cmista = widgets.FloatSlider(min=0.01,max=1,step=0.01,value=0.5,description='Proporção de asa reta'),
        cl     = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.25,description='Linha de referência'),
        );
widgets.HBox([Srefmista,ARmista,CMAmista])

interactive(children=(FloatSlider(value=11.0, description='Envergadura [m]', max=35.0, min=0.5, step=0.01), Fl…

## Asa Elíptica

A asa elíptica é definida por:

* $B$ - Envergadura
* $Cr$ - Corda da raíz (Root chord)
* $\lambda$ - Afilamento (Taper ratio)
* $\Lambda$ - Enflechamento (Sweept angle)
* $cl$ - Linha de centro

### Exemplo:

![Spitfire](https://i.pinimg.com/originals/0a/37/51/0a3751ced51a7a72f4a43c31f7d2e96d.jpg)



In [9]:
Srefelipt= widgets.BoundedFloatText(description='Área de asa [m²]:',format='0.1f',disabled=True)
ARelipt  = widgets.BoundedFloatText(description='Alongamento:',disabled=True)
CMAelipt = widgets.BoundedFloatText(description='CMA:',disabled=True)

def AsaEliptica(B,Croot,taper,sweept,cl):
    
    def Cb(x,b,c):
        return math.pow(np.interp(x,b,c),2)
    
    Nchords = 10
    t = np.linspace(0,np.pi/2,Nchords)
    b = 0 +(B/2*np.sin(t))
    
    c = Croot*taper +(Croot*(1-taper)*np.cos(t))   
    
    #for bi in b:
    #    if bi <= cmista*B/2:
    #        c.append(Croot)
    #    else:
    #        if taper <= 1:
    #            c.append(np.interp(bi,[cmista*B/2, B/2],[Croot, Croot*taper]))
    #        else:
    #            c.append(np.interp(bi,np.flip([cmista*B/2, B/2]),np.flip([Croot, Croot*taper])))
    
    I = quad(Cb,0.1,B/2,args=(b,c),limit=50,epsrel=0.001)
    clelipt=[]
    LEelipt=[]
    TEelipt=[]
    LEelipt.append(0)
    TEelipt.append(Croot)
    clelipt.append(Croot*cl)

    
    for chord,span in zip(c[1:],b[1:]):
        clelipt.append(Croot*cl+span*math.tan(sweept*np.pi/180))
        LEelipt.append(Croot*cl+span*math.tan(sweept*np.pi/180)-chord*cl)
        TEelipt.append(Croot*(cl)+span*math.tan(sweept*np.pi/180)+chord*(1-cl)) 
    
    Srefelipt.value = trapz(c,b)
    
    
    ARelipt.value   = (B*B)/Srefelipt.value
    CMAelipt.value  = max(I)/Srefelipt.value
    
    if taper <= 1:
        bcma = np.interp(CMAelipt.value,np.flip(c),np.flip(b))
    else:
        bcma = np.interp(CMAelipt.value,c,b)
    LEcma= np.interp(bcma,b,LEelipt)
    TEcma= np.interp(bcma,b,TEelipt)
    
    fig = plt.figure(figsize=(15, 15), dpi=72, facecolor='w', edgecolor='k')
    ax = fig.add_subplot(2, 1, 1)
    ax.plot(b, LEelipt, color="black")
    ax.plot(b, TEelipt, color="black")
    ax.plot(-b,LEelipt, color="black")
    ax.plot(-b,TEelipt, color="black")  
    ax.plot([B/2,B/2],[LEelipt[-1],TEelipt[-1]], color="black")
    ax.plot([-B/2,-B/2],[LEelipt[-1],TEelipt[-1]], color="black")
    
    ax.plot([bcma, bcma],[LEcma, TEcma], color="red")
    ax.plot(-b,clelipt,color="gray")
    ax.plot(b, clelipt,color="gray")
    ax.grid(True)
    ax.title.set_text('Asa elíptica')
    ax.set_xlabel('Envergadura [m]')
    ax.set_ylabel('Corda [m]')
    ax.set_aspect('equal', 'datalim')
    fig.canvas.draw()
    
interact(AsaEliptica,
        B      = widgets.FloatSlider(min=0.5,max=35.0,step=0.01,value=12.0, description='Envergadura [m]'),
        Croot  = widgets.FloatSlider(min=0.22,max=10.0,step=0.01, value=4.5, description='Corda na raíz[m]'),
        taper  = widgets.FloatSlider(min=0.1,max=2.0, step=0.01,value=0.1, description='Afilamento'),
        sweept = widgets.FloatSlider(min=-15,max=50,step=0.01,value=0,description='Enflechamento [º]'),
        cl     = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.25,description='Linha de referência'),
        );
widgets.HBox([Srefelipt,ARelipt,CMAelipt])

interactive(children=(FloatSlider(value=12.0, description='Envergadura [m]', max=35.0, min=0.5, step=0.01), Fl…

# Desafio!

Repararam como as funções são muito similares? o que acha de juntar todos os tipos de asas em uma função só?